# Homework 5

In this homework, you'll review some linear algebra, and practice using `scipy.optimize` to solve several variations of linear models.  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import scipy

## Linear Models

A standard problem in statistics to solve the multivariate linear regression problem.
\begin{equation}
y = X * \beta + \epsilon
\end{equation}
The above notation is standard in statistics, but in our discussion (and codes) we will replace $\beta$ with `b`
```
y = X * b + eps.
```
`X` is known as the [design matrix](https://en.wikipedia.org/wiki/Design_matrix), and consists of `n` rows of observations, each of which has `p` features (so it is an $n\times p$ matrix).  `y` is a vector of `n` responses.  `b` is an unknown vector of `p` coefficients which we would like to find.  `eps` (epsilon) is a vector of length `n` with random noise, typically i.i.d. normally distributed with variance `sig` (sigma).

In numpy notation, we could express this as
```python
y[i] = np.dot(X[i], b) + sig * np.random.randn()
```

We want to determine `b`, so that when me make a new observation `X[n]` we can predict the response `y[n]`.  One way to do this is to minimize the mean square error

\begin{equation}
\mathop{\mathsf{minimize}}_b \mathbb{E}((X[n]*b - y[n])^2)
\end{equation}

The solution to this is the solution to the least squares problem
\begin{equation}
\mathop{\mathsf{minimize}}_b \frac{1}{n} \|X*b - y\|_2^2
\end{equation}

Where $n$ is the number of rows in $X$.  We'll let the solution to the problem be denoted $\hat{b}$, or `bhat`.

## Problem 0 - Linear Regression (37 points)

There are a variety of ways to solve the above problem.  We will assume that $n > p$, meaning there are more observations than features.

### Part A: Generating Data (5 points)

Write a function `gen_lstsq(n, p, sig=0.1)` which will generate a linear least squares problem (return `X`, `y`, and `b` as described above).  `b` is the "ground truth" coefficients that we are looking for.  You can generate the matrix `X` using `np.random.randn`, and generate `b` using `np.random.randn` as well. 


### Part B: QR factorization (3 points)

If we form a QR factorization $X = QR$, we can find $\hat{b} = R^{-1} Q^T y$.

Write a function `solve_lstsq_qr(X, y)` which estimates `b` using the QR factorization as above. 

### Part C: Normal Equations (3 points)

Often, this is the way statistics textbooks solve the problem: $\hat{b} = (X^T X)^{-1} X^T y$.  This is based on the normal equation $X^T X \hat{b} = X^T y$.

Write a function `solve_lstsq_normal(X, y)` which estimates `b` using the normal equations above. Note that forming the full inverse would be slow and unstable. Instead, you should try to apply $(X^TX)^{-1}$ by forming a Cholesky factorization instead.

### Part D: Check Your Work (3 points)

Generate a few random problems to test that `solve_lstsq_qr` and `solve_lstsq_normal` give the same prediction $\hat{b}$ (measure $\|\hat{b}_{qr} - \hat{b}_{normal}\|_2$ and check it is smaller than `1e-4`).  Use $n > p$.  Check against `solve_lstsq` in numpy or scipy as well.

### Part E: Estimate the Error (6 points)

Write a function `err` which estimates the error in your fit using the equation $\frac{1}{n}\|X * \hat{b} - y\|_2^2$. The function should have the call signature `err(X,y,bhat).` 

Create a plot of the error vs the noise parameter `sig` (i.e. use your code from the previous parts to generate $y,$ $\hat{b},$ and $X$ for various $\sigma \in[10^{-4},10]$ and use `err` to compute the corresponding error).  Use `n=100`, `p=50` when generating data.  Put your plot on log-log axes.  Give it a title and axis labels.

As above, you should generate `X` to be `N x p` using `np.random.randn`, and set `y = X * b + sig * np.random.randn(p)`. Note: when choosing `sig` on $[10^{-4},10]$ it would help if you gave them a logarithmic distribution (like taking equispaced points $t_0,\dots,t_N$ between -4 and 1, and setting $\sigma_0 =10^{t_0},\dots, \sigma_N = 10^{t_n}.$

Try to make smoother plots by averaging the error over several trials.


### Part F: Discussion (10 points)

Which of `solve_lstsq_qr` and `solve_lstsq_normal` is faster?  Give a justification for your answer (e.g. in terms of what you know about time to compute matrix multiplication and factorizations as a function of $n$ and $p$).

Suppose that $\kappa(X)$ is large. Which of `solve_lstsq_qr` and `solve_lstsq_normal` will be more stable? i.e. which of $\hat{b} = R^{-1} Q^T y$ and $\hat{b} = (X^T X)^{-1} X^T y$ has a smaller condition number? Use the fact that the matrix condition number satisfies
$$
\kappa(A^{-1}) = \kappa(A), \qquad \kappa(A^T) = \kappa(A), \qquad \kappa(AB) \approx \kappa(A)\kappa(B),
$$
and that
$$
\kappa(Q) = 1
$$
because the columns of $Q$ are orthonormal. You should also use the fact that for most $A$ and $B$
$$
\kappa(AB) \approx \kappa(A)\kappa(B).
$$
Note that there are counter examples, e.g. $1=\kappa(I)=\kappa(AA^{-1})< \kappa(A)\kappa(A^{-1})=\kappa(A)^2$ whenever $\kappa(A)>1$.

### Part G: Optimization (5 points)

Solve the minimization problem 
\begin{equation}
\mathop{\mathsf{minimize}}_b \frac{1}{n}\|X*b - y\|_2^2
\end{equation}

using `scipy.optimize.minimize`.  Wrap this in a function `solve_lstsq_opt(X, y)`.  Note that you can define the objective function inside `solve_lstsq_opt`

**Jacobian**: you should implement the Jacobian of the objective function to use in the optimization problem.  Here's a derivation:  We'd like to minimize the objective function
\begin{equation}
n f(b) = \|X*b - y\|_2^2 = (Xb - y)^T (Xb - y) = b^T X^T X b - 2 y^T X b + y^T y
\end{equation}

We might write the above expression as
\begin{equation}
n f(b) \sum_{i,j} b_i (X^T X)_{i,j} b_j - 2\sum_{j,i} y_i X_{i,j} b_j + y^T y
\end{equation}

We can take a derivative with respect to $b_j$
\begin{equation}
n \frac{\partial f}{\partial b_j} = \sum_{i\ne j} b_i (X^T X)_{i,j} + \sum_{i\ne j} (X^T X)_{j,i} b_i + 2 (X^T X)_{j,j} b_j  - 2\sum_{i} y_i X_{i,j}
\end{equation}

Putting this in matrix form, we obtain
\begin{equation}
J_f(b) =  \frac{1}{n}\big( b^T (X^T X) + b^T (X^T X)^T - 2y^T X\big) = \frac{2}{n} b^T (X^T X) -\frac{2}{n}y^T X
\end{equation}

So we can write $J_f(b) = \frac{2}{n} b^T (X^T X) -\frac{2}{n} y^T X$

#### Check Your Work

Generate a few random problems to test that `solve_lstsq_opt` agrees with `solve_lstsq_qr` and `solve_lstsq_normal` in part A.  Use the same tolerance for checking.

### Part H: Discussion (2 points)

How fast is `solve_lstsq_opt` compared to the functions you wrote in part A?  Give some justification for what you see.

## Problem 1 - Ridge Regression (13 points)

We'll now turn to the problem of what to do when `n < p` (there are now fewer observations than features).  In this case we can find a $b$ which satisfies the equation $X * b = y$ exactly, but there are many possible values of $b$ which can satisfy the equation.

As we discussed in class, we can change the problem to try to make it unique. One simple way of doing this is to add an additional term to our optimization problem. Ridge regression is one such approach.

Ridge regression seeks to solve the following optimization problem:

\begin{equation}
\mathop{\mathsf{minimize}}_b \frac{1}{n}\|X*b - y\|_2^2 + \lambda \|b\|_2^2
\end{equation}

$\lambda$ is a parameter you can choose to "regularize" the problem which you can choose.

### Part A: Optimization (9 points)

Use `scipy.optimize.minimize` to minimize the above optimization problem.  Wrap this in a function `solve_ridge_opt(X, y, lam=0.1)` (`lam` should be provided as a keyword argument for the variable $\lambda$).

What is the Jacobian for the objective function for the minimization problem?  Incorporate this into your optimization problem.  Hint: differentiation is linear, so you just need to add a term to the Jacobian in Problem 0 based on the Jacobian of
\begin{equation}
\lambda \|b\|_2^2 = \lambda b^T b = \lambda \sum_i b_i^2
\end{equation}

---


### Part B: Compute the error (4 points)

Set `n = 50`, `p=100`, and `sig=0.1`.  Make a plot that displays the error of `bhat` computed using `solve_ridge_opt` as `lam` varies between `1e-4` and `1e2` (see Problem 0, Part E).  Use a `semilogx` plot.

### Part C: Analysis (optional - no points)

Find an analytic expression for the true solution to the ridge regression problem in terms of $X,$ $\lambda,$ and $y.$

### Part D: SVD (optional - no points)

Use the SVD to solve $X*b = y.$ Do this by forming the SVD of $X = U\Sigma V^T.$ Set $b = V z$ where $V$ is the $p\times n$ matrix consisting of the first $n$ right singular vectors (why?). Then, the system you get is $U \Sigma z = y.$ You can solve it by applying $U^T$ to both sides (why?) and noting that $\Sigma$ is diagonal. 

Note that you should be using the version of the SVD which (for $n<p$ but $X$ having full rank) gives $U$ as an $n\times n$ matrix, $\Sigma$ as an $n\times n$ diagonal matrix, and $V$ as a $p \times n$ matrix.